In [1]:
"""
Test Scenario Generator for Integrated Solution

This script provides an end-to-end solution that:
1. Finds the latest integrated_solution folder
2. Analyzes each US_[number]_code.py file to create detailed descriptions
3. Saves these descriptions in JSON format
4. Reads the original technical specifications from the Excel file
5. Combines code descriptions, tech specs, and user stories to generate test scenarios
6. Saves the test scenarios in both JSON and Markdown formats

The workflow is completely automated from integrated solution to test scenarios.
"""

import os
import logging
import json
import re
import pandas as pd
import ast
import textwrap
from typing import TypedDict, Annotated, List, Dict, Tuple, Optional, Set, Any
import operator
from langgraph.graph import StateGraph, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage
from datetime import datetime
from glob import glob
import shutil

from langchain_openai import AzureChatOpenAI

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s'
)
logger = logging.getLogger(__name__)

# Azure OpenAI configuration
def check_openai_config():
    """Check if Azure OpenAI config is set in environment variables."""
    required_vars = [
        "AZURE_OPENAI_API_KEY",
        "AZURE_OPENAI_ENDPOINT",
        "AZURE_OPENAI_API_VERSION",
        "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"
    ]
    
    missing = [var for var in required_vars if not os.environ.get(var)]
    if missing:
        # Check if we can set values explicitly
        logger.info("Looking for OpenAI configuration in current environment...")
        
        if "AZURE_OPENAI_API_KEY" not in os.environ:
            os.environ["AZURE_OPENAI_API_KEY"] = "0bf3daeba1814d03b5d62e1da4077478"
        
        if "AZURE_OPENAI_ENDPOINT" not in os.environ:
            os.environ["AZURE_OPENAI_ENDPOINT"] = "https://openaisk123.openai.azure.com/"
        
        if "AZURE_OPENAI_API_VERSION" not in os.environ:
            os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
        
        if "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME" not in os.environ:
            os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-4o"
    
    # Verify all variables are set
    missing = [var for var in required_vars if not os.environ.get(var)]
    if missing:
        raise EnvironmentError(f"Missing Azure OpenAI configuration: {', '.join(missing)}")
    
    logger.info("Azure OpenAI configuration verified")

#############################################################
# Part 1: Code Description Generation
#############################################################

class CodeDescriptionState(TypedDict):
    """State management for code description process"""
    messages: Annotated[list[AnyMessage], operator.add]
    current_description: dict
    user_story_id: str
    file_path: str

# Define prompts for code description
code_analyzer_prompt = """
Role: Python Code Analyst
Task: Generate a detailed description of the provided Python code.

Instructions:
1. Analyze the provided code and create a comprehensive description of its functionality, components, and architecture.
2. Focus on the following aspects:
   - Overall purpose and functionality
   - Key classes and their responsibilities
   - Main functions and their roles
   - Data structures used
   - Input/output handling
   - Error handling mechanisms
   - Any notable algorithms or patterns
   - Dependencies and external libraries
   - Configuration and settings

3. Organize your analysis in a structured manner, with clear sections for each major component.

Code to analyze:
```python
{code}
```

## Output Format
Your response must be a valid JSON object with the following structure:

```json
{{
  "module_name": "{module_name}",
  "overall_purpose": "A concise description of what this code does",
  "architecture": {{
    "description": "Overall architectural approach",
    "patterns_used": ["pattern1", "pattern2"]
  }},
  "key_components": [
    {{
      "name": "ComponentName",
      "type": "class/function",
      "purpose": "What this component does",
      "functionality": "Detailed description of how it works"
    }}
  ],
  "data_flow": "Description of how data moves through the system",
  "input_handling": "How inputs are processed",
  "output_handling": "How outputs are generated",
  "error_handling": "Error handling approach",
  "dependencies": ["dependency1", "dependency2"],
  "notable_algorithms": [
    {{
      "name": "Algorithm name",
      "purpose": "What it accomplishes",
      "description": "How it works"
    }}
  ],
  "configuration": "How the code is configured",
  "assumptions": ["assumption1", "assumption2"],
  "limitations": ["limitation1", "limitation2"]
}}
```

Remember to make your JSON valid. Escape quotes within strings and ensure the structure matches exactly what is requested.
"""

def find_latest_integrated_solution_folder(base_dir=None):
    """Find the latest integrated_solution folder based on creation time."""
    if base_dir is None:
        base_dir = os.getcwd()  # Current working directory
    
    integrated_folders = [d for d in os.listdir(base_dir) if d.startswith("integrated_solution_") and os.path.isdir(os.path.join(base_dir, d))]
    if not integrated_folders:
        raise FileNotFoundError("No integrated_solution folders found")
    
    # Sort by creation time, most recent first
    integrated_folders.sort(key=lambda d: os.path.getctime(os.path.join(base_dir, d)), reverse=True)
    return os.path.join(base_dir, integrated_folders[0])

def find_code_files(solution_folder):
    """Find all US_[number]_code.py files in the integrated solution folder."""
    pattern = os.path.join(solution_folder, "US_*_code.py")
    return glob(pattern)

def create_output_directory():
    """Create output directory for code descriptions and test scenarios."""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir = f"test_scenarios_{timestamp}"
    
    # Create main directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Create subdirectories
    descriptions_dir = os.path.join(output_dir, "code_descriptions")
    test_scenarios_dir = os.path.join(output_dir, "test_scenarios")
    
    os.makedirs(descriptions_dir, exist_ok=True)
    os.makedirs(test_scenarios_dir, exist_ok=True)
    
    return output_dir, descriptions_dir, test_scenarios_dir

def extract_user_story_id(file_path):
    """Extract user story ID from filename."""
    filename = os.path.basename(file_path)
    match = re.match(r'US_(\d+)_code\.py', filename)
    if match:
        return f"US_{match.group(1)}"
    return None

class CodeDescriptionGenerator:
    """Main class for generating detailed code descriptions"""
    
    def __init__(self, model, output_dir, system_analyzer=""):
        self.system_analyzer = system_analyzer
        self.output_dir = output_dir
        
        # Initialize graph
        graph = StateGraph(CodeDescriptionState)
        
        # Add nodes
        graph.add_node("analyzer", self.analyzer)
        
        # Add edges
        graph.add_edge("analyzer", END)
        
        # Set entry point
        graph.set_entry_point("analyzer")
        self.graph = graph.compile()
        self.model = model
    
    def analyzer(self, state: CodeDescriptionState):
        """Generate code description"""
        messages = state['messages']
        user_story_id = state.get('user_story_id', 'unknown_id')
        file_path = state.get('file_path', '')
        
        logger.info(f"Analyzing code for {user_story_id} from {file_path}")
        
        if self.system_analyzer:
            # Read code file
            with open(file_path, 'r') as f:
                code = f.read()
            
            module_name = os.path.basename(file_path)
            
            # Format the prompt
            formatted_prompt = self.system_analyzer.format(
                code=code,
                module_name=module_name
            )
            messages = [SystemMessage(content=formatted_prompt)]
        
        message = self.model.invoke(messages)
        response_text = getattr(message, "content", "")
        
        # Extract JSON from response
        description = self._extract_json_from_response(response_text, user_story_id)
        
        # Save description
        self._save_description(description, user_story_id)
        
        return {
            'messages': [message],
            'current_description': description,
            'user_story_id': user_story_id,
            'file_path': file_path
        }
    
    def _extract_json_from_response(self, response_text, user_story_id):
        """Extract JSON from the LLM response."""
        try:
            # Try to find JSON block first
            json_pattern = r"```json\s*(.*?)\s*```"
            match = re.search(json_pattern, response_text, re.DOTALL)
            if match:
                json_str = match.group(1)
                return json.loads(json_str)
            
            # If no JSON block, try to parse the whole response
            return json.loads(response_text)
        except Exception as e:
            logger.error(f"Failed to extract JSON for {user_story_id}: {e}")
            
            # Create a fallback description
            return {
                "module_name": user_story_id,
                "overall_purpose": "Could not extract description",
                "error": str(e),
                "raw_response": response_text
            }
    
    def _save_description(self, description, user_story_id):
        """Save code description to JSON file."""
        file_path = os.path.join(self.output_dir, f"{user_story_id}_description.json")
        with open(file_path, 'w') as f:
            json.dump(description, f, indent=2)
        
        logger.info(f"Saved code description to {file_path}")

def process_code_files(code_files, descriptions_dir):
    """
    Process all code files to generate descriptions
    
    Args:
        code_files: List of paths to code files
        descriptions_dir: Directory to save descriptions
        
    Returns:
        Dict mapping user story IDs to their descriptions
    """
    # Initialize OpenAI model
    check_openai_config()
    
    model = AzureChatOpenAI(
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        api_version=os.environ["AZURE_OPENAI_API_VERSION"],
        deployment_name=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"]
    )
    
    # Initialize code description generator
    code_desc_gen = CodeDescriptionGenerator(
        model=model,
        output_dir=descriptions_dir,
        system_analyzer=code_analyzer_prompt
    )
    
    descriptions = {}
    
    # Process each code file
    for file_path in code_files:
        user_story_id = extract_user_story_id(file_path)
        if not user_story_id:
            logger.warning(f"Could not extract user story ID from {file_path}, skipping")
            continue
        
        logger.info(f"Processing code file for {user_story_id}")
        
        # Setup initial state
        initial_state = {
            "messages": [],
            "current_description": {},
            "user_story_id": user_story_id,
            "file_path": file_path
        }
        
        try:
            # Run the graph
            result = code_desc_gen.graph.invoke(initial_state)
            
            # Store description
            if 'current_description' in result and result['current_description']:
                descriptions[user_story_id] = result['current_description']
                logger.info(f"Successfully generated description for {user_story_id}")
        
        except Exception as e:
            logger.error(f"Error processing code file for {user_story_id}: {e}")
            continue
    
    return descriptions

#############################################################
# Part 2: Test Case Generation
#############################################################

class TestScenarioState(TypedDict):
    """State management for test scenario generation process"""
    messages: Annotated[list[AnyMessage], operator.add]
    current_test_scenarios: dict
    user_story_id: str
    code_description: dict
    tech_spec: str
    user_story: str

# Define prompts for test scenario generation
test_scenario_generator_prompt = """
Role: Test Scenario Designer
Task: Generate test scenarios for a software module based on its description, technical specifications, and user story.

Provided Information:
- Code Description: A detailed analysis of the module's functionality and components
- Technical Specification: The original technical requirements that guided development
- User Story: The business need that the code addresses

Instructions:
1. Analyze all provided information to understand the module's purpose, functionality, and requirements.
2. Design test scenarios that cover:
   - Main functionality (core features and capabilities)
   - Alternative flows (different ways to achieve the same goal)
   - Edge cases (extreme or unusual situations)
   - Error handling (how the system should respond to invalid inputs)
   - Integration points (how it connects with other components)

3. For each test scenario, provide:
   - A unique identifier
   - A descriptive name
   - A brief description of the scenario context
   - What should be tested
   - The expected outcome
   - Test category (e.g., functional, edge case, error, integration)

Code Description:
{code_description}

Technical Specification:
{tech_spec}

User Story:
{user_story}

## Output Format
Your response must be a valid JSON object with the following structure:

```json
{{
  "module_id": "{user_story_id}",
  "test_suite_name": "Test Scenarios for {user_story_id}",
  "summary": "A brief overview of the test approach",
  "test_scenarios": [
    {{
      "id": "TS-{user_story_id}-001",
      "name": "Scenario descriptive name",
      "category": "functional|edge_case|error|integration|performance",
      "description": "A description of the scenario context and what is being tested",
      "test_objective": "What specific aspect or functionality is being verified",
      "expected_outcome": "What should happen if the system works correctly",
      "relevant_requirements": "References to specific requirements or user story elements"
    }},
    {{
      "id": "TS-{user_story_id}-002",
      "name": "Another test scenario",
      ...
    }}
  ],
  "coverage": {{
    "functional_areas": ["area1", "area2"],
    "edge_cases": ["case1", "case2"],
    "not_covered": ["limitation1"]
  }}
}}
```

Generate at least 5-8 test scenarios that together provide good coverage of the module's functionality. Focus on capturing the essence of what needs to be tested rather than detailed test steps.
"""

def read_tech_specs_and_user_stories(excel_file_path):
    """Read technical specifications and user stories from Excel file."""
    try:
        # Read the Excel file
        df = pd.read_excel(excel_file_path)
        
        # Find the user story column and tech spec column
        user_story_col = None
        tech_spec_col = None
        
        # Determine column names - assuming first row has column headers
        col_names = df.columns.tolist()
        
        # Find user story column
        for col in col_names:
            if 'user' in str(col).lower() and 'story' in str(col).lower():
                user_story_col = col
                break
        
        # Find tech spec column
        for col in col_names:
            if ('tech' in str(col).lower() and 'spec' in str(col).lower()) or 'requirement' in str(col).lower():
                tech_spec_col = col
                break
        
        # If we didn't find the right columns, default to the first two
        if user_story_col is None and len(col_names) > 0:
            user_story_col = col_names[0]
        
        if tech_spec_col is None and len(col_names) > 1:
            tech_spec_col = col_names[1]
        
        logger.info(f"Using columns: User Story = '{user_story_col}', Tech Spec = '{tech_spec_col}'")
        
        # Extract user stories and tech specs
        user_stories = {}
        tech_specs = {}
        
        # Skip the first row if it's empty
        start_row = 1 if df.iloc[0].isna().all() else 0
        
        for idx, row in df.iloc[start_row:].iterrows():
            if pd.isna(row[user_story_col]) or pd.isna(row[tech_spec_col]):
                logger.warning(f"Skipping row {idx} due to missing data")
                continue
                
            user_story_text = str(row[user_story_col])
            tech_spec_text = str(row[tech_spec_col])
            
            # Extract user story ID
            match = re.search(r'User\s+Story\s+ID\s*:\s*(\d+)', user_story_text, re.IGNORECASE)
            if match:
                user_story_id = f"US_{match.group(1)}"
                user_stories[user_story_id] = user_story_text
                tech_specs[user_story_id] = tech_spec_text
            else:
                # Alternative pattern
                match = re.search(r'^(?:(?:user)?story|us)(\d+)', user_story_text, re.IGNORECASE | re.MULTILINE)
                if match:
                    user_story_id = f"US_{match.group(1)}"
                    user_stories[user_story_id] = user_story_text
                    tech_specs[user_story_id] = tech_spec_text
                else:
                    logger.warning(f"Could not extract user story ID from: {user_story_text[:50]}...")
        
        logger.info(f"Successfully extracted {len(user_stories)} user stories and tech specs from Excel file")
        return user_stories, tech_specs
        
    except Exception as e:
        logger.error(f"Error reading Excel file: {e}")
        return {}, {}

class TestScenarioGenerator:
    """Main class for generating test scenarios"""
    
    def __init__(self, model, output_dir, system_generator=""):
        self.system_generator = system_generator
        self.output_dir = output_dir
        
        # Initialize graph
        graph = StateGraph(TestScenarioState)
        
        # Add nodes
        graph.add_node("generator", self.generator)
        
        # Add edges
        graph.add_edge("generator", END)
        
        # Set entry point
        graph.set_entry_point("generator")
        self.graph = graph.compile()
        self.model = model
    
    def generator(self, state: TestScenarioState):
        """Generate test scenarios"""
        messages = state['messages']
        user_story_id = state.get('user_story_id', 'unknown_id')
        code_description = state.get('code_description', {})
        tech_spec = state.get('tech_spec', '')
        user_story = state.get('user_story', '')
        
        logger.info(f"Generating test scenarios for {user_story_id}")
        
        if self.system_generator:
            # Format the prompt
            formatted_prompt = self.system_generator.format(
                code_description=json.dumps(code_description, indent=2),
                tech_spec=tech_spec,
                user_story=user_story,
                user_story_id=user_story_id
            )
            messages = [SystemMessage(content=formatted_prompt)]
        
        message = self.model.invoke(messages)
        response_text = getattr(message, "content", "")
        
        # Extract JSON from response
        test_scenarios = self._extract_json_from_response(response_text, user_story_id)
        
        # Save test scenarios
        self._save_test_scenarios(test_scenarios, user_story_id)
        
        return {
            'messages': [message],
            'current_test_scenarios': test_scenarios,
            'user_story_id': user_story_id,
            'code_description': code_description,
            'tech_spec': tech_spec,
            'user_story': user_story
        }
    
    def _extract_json_from_response(self, response_text, user_story_id):
        """Extract JSON from the LLM response."""
        try:
            # Try to find JSON block first
            json_pattern = r"```json\s*(.*?)\s*```"
            match = re.search(json_pattern, response_text, re.DOTALL)
            if match:
                json_str = match.group(1)
                return json.loads(json_str)
            
            # If no JSON block, try to parse the whole response
            return json.loads(response_text)
        except Exception as e:
            logger.error(f"Failed to extract JSON for test scenarios {user_story_id}: {e}")
            
            # Create a fallback structure
            return {
                "module_id": user_story_id,
                "test_suite_name": f"Test Scenarios for {user_story_id}",
                "summary": "Error extracting test scenarios",
                "test_scenarios": [],
                "error": str(e),
                "raw_response": response_text
            }
    
    def _save_test_scenarios(self, test_scenarios, user_story_id):
        """Save test scenarios to JSON and MD files."""
        # Save as JSON
        json_path = os.path.join(self.output_dir, f"{user_story_id}_test_scenarios.json")
        with open(json_path, 'w') as f:
            json.dump(test_scenarios, f, indent=2)
        
        # Save as Markdown
        md_path = os.path.join(self.output_dir, f"{user_story_id}_test_scenarios.md")
        with open(md_path, 'w') as f:
            f.write(self._convert_to_markdown(test_scenarios))
        
        logger.info(f"Saved test scenarios to {json_path} and {md_path}")
    
    def _convert_to_markdown(self, test_scenarios):
        """Convert test scenarios JSON to Markdown format."""
        md_content = []
        
        # Add header
        md_content.append(f"# {test_scenarios.get('test_suite_name', 'Test Scenarios')}")
        md_content.append("")
        
        # Add summary
        md_content.append("## Summary")
        md_content.append(test_scenarios.get('summary', 'No summary provided.'))
        md_content.append("")
        
        # Add test coverage
        coverage = test_scenarios.get('coverage', {})
        if coverage:
            md_content.append("## Test Coverage")
            
            # Functional areas
            functional = coverage.get('functional_areas', [])
            if functional:
                md_content.append("### Functional Areas")
                for area in functional:
                    md_content.append(f"- {area}")
                md_content.append("")
            
            # Edge cases
            edge_cases = coverage.get('edge_cases', [])
            if edge_cases:
                md_content.append("### Edge Cases")
                for case in edge_cases:
                    md_content.append(f"- {case}")
                md_content.append("")
            
            # Not covered
            not_covered = coverage.get('not_covered', [])
            if not_covered:
                md_content.append("### Areas Not Covered")
                for area in not_covered:
                    md_content.append(f"- {area}")
                md_content.append("")
        
        # Add test scenarios
        md_content.append("## Test Scenarios")
        md_content.append("")
        
        for ts in test_scenarios.get('test_scenarios', []):
            md_content.append(f"### {ts.get('id', 'TS-XXX')}: {ts.get('name', 'Unnamed Test Scenario')}")
            md_content.append("")
            
            md_content.append(f"**Category:** {ts.get('category', 'Unknown')}")
            md_content.append("")
            
            md_content.append("**Description:**")
            md_content.append(ts.get('description', 'No description provided.'))
            md_content.append("")
            
            md_content.append("**Test Objective:**")
            md_content.append(ts.get('test_objective', 'No objective provided.'))
            md_content.append("")
            
            md_content.append("**Expected Outcome:**")
            md_content.append(ts.get('expected_outcome', 'No expected outcome provided.'))
            md_content.append("")
            
            # Relevant requirements
            relevant_requirements = ts.get('relevant_requirements', '')
            if relevant_requirements:
                md_content.append(f"**Relevant Requirements:** {relevant_requirements}")
                md_content.append("")
        
        return "\n".join(md_content)

def generate_test_scenarios(descriptions, user_stories, tech_specs, test_scenarios_dir):
    """
    Generate test scenarios for each module
    
    Args:
        descriptions: Dict mapping user story IDs to their code descriptions
        user_stories: Dict mapping user story IDs to user story texts
        tech_specs: Dict mapping user story IDs to tech spec texts
        test_scenarios_dir: Directory to save test scenarios
        
    Returns:
        Dict mapping user story IDs to their test scenarios
    """
    # Initialize OpenAI model
    check_openai_config()
    
    model = AzureChatOpenAI(
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        api_version=os.environ["AZURE_OPENAI_API_VERSION"],
        deployment_name=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"]
    )
    
    # Initialize test scenario generator
    test_gen = TestScenarioGenerator(
        model=model,
        output_dir=test_scenarios_dir,
        system_generator=test_scenario_generator_prompt
    )
    
    test_scenarios = {}
    
    # Process each module
    for user_story_id, description in descriptions.items():
        # Get user story and tech spec if available
        user_story = user_stories.get(user_story_id, f"User story for {user_story_id} not available")
        tech_spec = tech_specs.get(user_story_id, f"Technical specification for {user_story_id} not available")
        
        logger.info(f"Generating test scenarios for {user_story_id}")
        
        # Setup initial state
        initial_state = {
            "messages": [],
            "current_test_scenarios": {},
            "user_story_id": user_story_id,
            "code_description": description,
            "tech_spec": tech_spec,
            "user_story": user_story
        }
        
        try:
            # Run the graph
            result = test_gen.graph.invoke(initial_state)
            
            # Store test scenarios
            if 'current_test_scenarios' in result and result['current_test_scenarios']:
                test_scenarios[user_story_id] = result['current_test_scenarios']
                logger.info(f"Successfully generated test scenarios for {user_story_id}")
        
        except Exception as e:
            logger.error(f"Error generating test scenarios for {user_story_id}: {e}")
            continue
    
    return test_scenarios

def create_summary_report(output_dir, descriptions, test_scenarios):
    """Create a summary report of all code descriptions and test scenarios."""
    report = {
        "generation_timestamp": datetime.now().isoformat(),
        "modules_analyzed": len(descriptions),
        "test_scenarios_generated": sum(len(ts.get('test_scenarios', [])) for ts in test_scenarios.values()),
        "modules": [
            {
                "id": module_id,
                "purpose": desc.get('overall_purpose', 'No purpose provided'),
                "scenario_count": len(test_scenarios.get(module_id, {}).get('test_scenarios', [])),
                "scenario_categories": list(set(ts.get('category', 'unknown') 
                                           for ts in test_scenarios.get(module_id, {}).get('test_scenarios', [])))
            }
            for module_id, desc in descriptions.items()
        ]
    }
    
    # Save as JSON
    report_path = os.path.join(output_dir, "summary_report.json")
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=2)
    
    # Save as Markdown
    md_report_path = os.path.join(output_dir, "summary_report.md")
    
    md_content = [
        "# Test Scenario Generation Summary Report",
        "",
        f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        f"**Modules Analyzed:** {report['modules_analyzed']}",
        f"**Total Test Scenarios:** {report['test_scenarios_generated']}",
        "",
        "## Module Summary",
        ""
    ]
    
    # Add table header
    md_content.append("| Module ID | Purpose | Scenario Count | Scenario Categories |")
    md_content.append("| --- | --- | --- | --- |")
    
    # Add table rows
    for module in report['modules']:
        purpose = module['purpose']
        if len(purpose) > 50:
            purpose = purpose[:47] + "..."
        
        categories = ", ".join(module['scenario_categories']) if module['scenario_categories'] else "None"
        
        md_content.append(f"| {module['id']} | {purpose} | {module['scenario_count']} | {categories} |")
    
    with open(md_report_path, 'w') as f:
        f.write("\n".join(md_content))
    
    logger.info(f"Created summary report at {report_path} and {md_report_path}")

#############################################################
# Main Workflow
#############################################################

def main(excel_file_path="tech.xlsx"):
    """
    Main workflow function that runs the test scenario generation process
    
    Args:
        excel_file_path: Path to Excel file with user stories and tech specs
        
    Returns:
        str: Path to the output directory
    """
    try:
        # Step 1: Configure OpenAI
        check_openai_config()
        
        # Step 2: Find the latest integrated solution folder
        solution_folder = find_latest_integrated_solution_folder()
        logger.info(f"Found latest integrated solution folder: {solution_folder}")
        
        # Step 3: Find all US_*_code.py files
        code_files = find_code_files(solution_folder)
        logger.info(f"Found {len(code_files)} code files")
        
        if not code_files:
            logger.error("No code files found")
            return None
        
        # Step 4: Create output directories
        output_dir, descriptions_dir, test_scenarios_dir = create_output_directory()
        logger.info(f"Created output directories: {output_dir}")
        
        # Step 5: Process code files to generate descriptions
        descriptions = process_code_files(code_files, descriptions_dir)
        logger.info(f"Generated {len(descriptions)} code descriptions")
        
        # Step 6: Read user stories and tech specs from Excel
        user_stories, tech_specs = read_tech_specs_and_user_stories(excel_file_path)
        logger.info(f"Read {len(user_stories)} user stories and tech specs from Excel")
        
        # Step 7: Generate test scenarios
        test_scenarios = generate_test_scenarios(descriptions, user_stories, tech_specs, test_scenarios_dir)
        logger.info(f"Generated test scenarios for {len(test_scenarios)} modules")
        
        # Step 8: Create summary report
        create_summary_report(output_dir, descriptions, test_scenarios)
        
        logger.info(f"Test scenario generation completed successfully. Output directory: {output_dir}")
        print(f"Test scenario generation completed successfully. Output directory: {output_dir}")
        
        return output_dir
        
    except Exception as e:
        logger.error(f"Error in main workflow: {e}")
        import traceback
        traceback.print_exc()
        raise

if __name__ == "__main__":
    # Execute the test scenario generation process
    output_dir = main("tech.xlsx")
    
    if output_dir:
        print(f"\nWorkflow completed successfully!")
        print(f"Test scenarios and code descriptions saved to: {output_dir}")
    else:
        print("\nWorkflow completed with errors. Check logs for details.")

2025-03-11 17:54:41,777 - __main__ - INFO - [1201286376.py:52] - Looking for OpenAI configuration in current environment...
2025-03-11 17:54:41,779 - __main__ - INFO - [1201286376.py:71] - Azure OpenAI configuration verified
2025-03-11 17:54:41,782 - __main__ - INFO - [1201286376.py:812] - Found latest integrated solution folder: c:\Users\ADITYANJ\Downloads\stuff\integrated_solution_20250311_022136
2025-03-11 17:54:41,784 - __main__ - INFO - [1201286376.py:816] - Found 7 code files
2025-03-11 17:54:41,791 - __main__ - INFO - [1201286376.py:824] - Created output directories: test_scenarios_20250311_175441
2025-03-11 17:54:41,794 - __main__ - INFO - [1201286376.py:71] - Azure OpenAI configuration verified
2025-03-11 17:54:41,898 - __main__ - INFO - [1201286376.py:319] - Processing code file for US_141
2025-03-11 17:54:41,922 - __main__ - INFO - [1201286376.py:219] - Analyzing code for US_141 from c:\Users\ADITYANJ\Downloads\stuff\integrated_solution_20250311_022136\US_141_code.py
2025-03

Test scenario generation completed successfully. Output directory: test_scenarios_20250311_175441

Workflow completed successfully!
Test scenarios and code descriptions saved to: test_scenarios_20250311_175441
